In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

## Prediction on Single Image & Plot with the predicted class

In [7]:
def prepare_test_image(image, img_shape = 224, scale = True):
    # Step 1 : Read the image
    test_img = tf.io.read_file(image)

    #Step 2: Decode the image in tensor
    test_img = tf.image.decode_image(test_img)

    #Step 3: Resize the image:
    test_img = tf.image.resize(test_img, size = [img_shape, img_shape])

    # Step 4: Rescale the image:
    if scale:
        test_img = test_img/255.
    else:
        test_img

    #Step 5: Convert into 4D
    test_img = tf.expand_dims(test_img, axis=0)
    
    return test_img

In [26]:
def predict_and_plot_output_single_testimage (test_image, model, class_names):
    """
    Preprocess the single test image
    
    Predict the output
    
    Finds out class name of the predicted output
    
    Plot the test image with predicted output class name
    """
    # Step 1 : Load and preprocess the test image
    testimage = prepare_test_image(test_image)

    # Step 2 : Predict
    prediction_output = model.predict(testimage)

    # Step 3 : Find out the predicted class
    if len(prediction_output[0]) > 1:
        pred_class = class_names[tf.argmax(prediction_output[0])]
    else:
        pred_class = class_names[int(tf.round(prediction_output[0]))]
        print("Class : ",pred_class)

    # Step 4 : Visualize the test image + PREDICTED CLASS
    plt.figure()
    plt.imshow(tf.squeeze(testimage))
    plt.title(f" Predicted Class : {pred_class}")
    plt.axis = False

## Creating TensorBoard Callback

#### For model comparisions on Tensorboard

In [32]:
import datetime

def create_tensorboard_callback(dir_name, experiment_name):
    log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
    print(f"Saving TensorBoard log files to: {log_dir}")
    return tensorboard_callback
    
    """
    Saves the log files to be used in tensorboard
    
    1 arguments needs to be added while fitting the model to enable this callback
    
    
    callbacks=[create_tensorboard_callback(dir_name="tensorflow_hub",experiment_name="resnet50V2")]
    """
    
checkpoint_path = "/content/custom_images_1to4/model_checkpoints/cp.ckpt"
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                      montior="val_acc",
                                                      save_best_only=True,
                                                      save_weights_only=True,
                                                      verbose=0) # don't print whether or not model is being saved
    
  

## Prepare Transfer Learning model + adjust o/p layer = no of classes

In [21]:
def prepare_transfer_learning_model(model_url, num_classes=10):
    feature_extractor_layer = hub.KerasLayer(model_url,
                                           trainable=False, # freeze the already learned patterns
                                           name="feature_extraction_layer",
                                           input_shape=(224,224)+(3,)) 
    model = tf.keras.Sequential([
    feature_extractor_layer,
    tf.keras.layers.Dense(num_classes, activation="softmax", name="output_layer")
  ])
    
    return model  

## Plotting Evaluation Curves - Accuracy & Loss (For Training & Validation data)

In [30]:
def plot_evaluation_curves(history):
    """
    For Training & Validation Data
    
    Training Loss vs Validation loss
    
    Training Accuracy vs Validation Loss
    """
    loss = resnet_history.history['loss']
    val_loss = resnet_history.history['val_loss']
    
    accuracy = resnet_history.history['accuracy']
    val_accuracy = resnet_history.history['val_accuracy']
    
    epochs = range(len(resnet_history.history["loss"]))
    
    ## Plotting loss curves
#     plt.figure()
    plt.plot(epochs, loss, label = "Training Loss")
    plt.plot(epochs, val_loss, label = "Validation loss")
    plt.title('Loss curves')
    plt.xlabel('Epochs')
    plt.legend()
    
    ## Plotting Accuracy curves
    plt.figure()   
    plt.plot(epochs, accuracy, label = 'Training Accuracy')
    plt.plot(epochs, val_accuracy, label = 'Validation Accuracy')
    plt.title("Accuracy Curves")
    plt.xlabel("Epochs")
    plt.legend()

## Predicting & Plotting Evaluation Curves - Accuracy & Loss (For Test data)

In [ ]:
# def predict_and_plot_testdata(model, test_data):
#     """
#     For complete testdataset
    
#     Loss()
#     """
#     model.evaluate(test_data)
#     df = pd.DataFrame(resnet_history.history)
#     df.plot(figsize=(10, 7))